In [29]:
import requests
import pandas as pd
from time import sleep
import os

### Testing to see response format

In [21]:
url = 'https://ukr.warspotting.net/api/losses/russia/1/'
json = requests.get(url, headers={'User-Agent': 'Chrome'}).json()

In [22]:
print(json)

{'losses': [{'id': 2, 'type': 'Tanks', 'model': 'T-64BV', 'status': 'Destroyed', 'lost_by': 'Russia', 'date': '2022-03-16', 'nearest_location': 'Mariupol, Mariupol raion', 'geolocation': '47.099125851628806,37.52371337039075', 'unit': None, 'oryx_url': 'https://i.postimg.cc/vTVpDRV2/4fhh.png'}, {'id': 3, 'type': 'Tanks', 'model': 'T-64BV', 'status': 'Destroyed', 'lost_by': 'Russia', 'date': '2022-03-16', 'nearest_location': 'Mariupol, Mariupol raion', 'geolocation': '47.09869256359657,37.52353235165147', 'unit': None, 'oryx_url': 'https://i.postimg.cc/nLkXCgJk/5gk.png'}, {'id': 5, 'type': 'Tanks', 'model': 'T-64BV', 'status': 'Destroyed', 'lost_by': 'Russia', 'date': '2022-03-17', 'nearest_location': 'Mariupol, Mariupol raion', 'geolocation': '47.098139835697424,37.640174323260645', 'unit': None, 'oryx_url': 'https://i.postimg.cc/13SPjpnQ/325.png'}, {'id': 6, 'type': 'Tanks', 'model': 'T-64BV', 'status': 'Destroyed', 'lost_by': 'Russia', 'date': '2022-03-31', 'nearest_location': 'Rubiz

In [25]:
df = pd.json_normalize(json, 'losses')

In [26]:
df

,id,type,model,status,lost_by,date,nearest_location,geolocation,unit,oryx_url
0,2,Tanks,T-64BV,Destroyed,Russia,2022-03-16,"Mariupol, Mariupol raion","47.099125851628806,37.52371337039075",None,https://i.postimg.cc/vTVpDRV2/4fhh.png
1,3,Tanks,T-64BV,Destroyed,Russia,2022-03-16,"Mariupol, Mariupol raion","47.09869256359657,37.52353235165147",None,https://i.postimg.cc/nLkXCgJk/5gk.png
2,5,Tanks,T-64BV,Destroyed,Russia,2022-03-17,"Mariupol, Mariupol raion","47.098139835697424,37.640174323260645",None,https://i.postimg.cc/13SPjpnQ/325.png
3,6,Tanks,T-64BV,Destroyed,Russia,2022-03-31,"Rubizhne, Severodonetsk raion","49.01122,38.39844",None,https://i.postimg.cc/Dm38Wx0v/669.png
4,8,Tanks,T-64BV,Destroyed,Russia,2022-03-07,"Mariupol, Mariupol raion","47.06449,37.47426",None,https://i.postimg.cc/Tw0Bjx5w/768.jpg
...,...,...,...,...,...,...,...,...,...,...
95,138,Infantry fighting vehicles,BMP-2(K),Damaged,Russia,2022-04-11,"Mariupol, Mariupol raion","47.09276,37.54362",None,https://twitter.com/UAWeapons/status/151349539...
96,143,Tanks,T-72B,Destroyed,Russia,2022-04-05,Chernihiv oblast,None,239th Tank Regiment,https://i.postimg.cc/KzgLWJ4m/341.png
97,145,Tanks,T-72B,Destroyed,Russia,2023-06-26,"Dovhenke, Izium raion","49.032611,37.316417",None,https://i.postimg.cc/Fzz1JcBV/1031-t72b-destr-...
98,147,Tanks,T-72B,Captured,Russia,2022-03-11,"Malyi Vystorop, Sumy raion","50.6277,34.81712",15th Motor Rifle Regiment,https://i.postimg.cc/Bv3rSysY/33213.png


### Get losses data and load into csv

In [46]:
output_path_losses = 'data/losses.csv'
base_url = 'https://ukr.warspotting.net/api/losses/russia/'

for id in range(1, 25514, 100):
    json = requests.get(f'{base_url}{id}/', headers={'User-Agent': 'Chrome'}).json()
    df = pd.json_normalize(json, 'losses')
    df.to_csv(output_path_losses, mode='a', index=False, header=not os.path.exists(output_path_losses))

In [47]:
# Drop duplicates returned by api not returning all unique ids
df = pd.read_csv('data/losses.csv')
print(f'{df.shape = }')
df_deduplicated = df.drop_duplicates()
print(f'{df_deduplicated.shape = }')

df.shape = (25579, 10)
df_deduplicated.shape = (14183, 10)


In [48]:
df_deduplicated.to_csv('data/losses.csv', index=False)

### Get stats data and load into csv

In [35]:
json = requests.get('https://ukr.warspotting.net/api/stats/russia/', headers={'User-Agent': 'Chrome'}).json()

,counts_by_type,counts_by_status.abandoned,counts_by_status.captured,counts_by_status.destroyed,counts_by_status.damaged
0,"[{'type_name': 'Tanks', 'counts': {'losses': '...",607,2804,10069,704


In [41]:
df = pd.json_normalize(json, 'counts_by_type')
df.head()

,type_name,counts.losses,counts.losses_new,counts.damaged,counts.damaged_new
0,Tanks,2513,7,136,0
1,Infantry fighting vehicles,4836,10,172,0
2,Infantry mobility vehicles,299,0,27,0
3,"Command posts, communication",335,0,4,0
4,Anti-tank systems,38,0,0,0


In [42]:
df.to_csv('data/stats.csv', index=False)